In [2]:
!pip install tensorflow_text==2.8.1

     |████████████████████████████████| 4.0 MB 594 kB/s eta 0:00:01
     |████████████████████████████████| 217.9 MB 86 kB/s  eta 0:00:01
     |████████████████████████████████| 5.8 MB 475 kB/s eta 0:00:01
     |████████████████████████████████| 462 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 2.2 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.0
    Uninstalling tensorboard-2.10.0:
      Successfully uninstalled tensorboard-2.10.0
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.10.0
    Uninstalling tensorflow-2.10.0:

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import pandas as pd

In [4]:
dataFrame=pd.read_csv('/Users/Bobbie/Downloads/BERT/spam.csv',encoding='ISO8859-1')

In [5]:
dataFrame.head(10)

,Category,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [6]:
dataFrame['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [7]:
dataFrame['Identifier']=dataFrame['Category'].apply(lambda x:1 if x=='spam' else 0)

In [8]:
dataFrame.head()

,Category,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4,Identifier
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,0
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,1
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,0


In [9]:
preprocess_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

model_url='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [10]:
bert_preprocess_url=hub.KerasLayer(preprocess_url)

2022-09-15 15:24:33.465346: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
bert_model_url=hub.KerasLayer(model_url)

In [12]:
myinputs=tf.keras.layers.Input(shape=(),dtype=tf.string,name='InputLayer')

preprocess_text=bert_preprocess_url(myinputs)

myoutputs=bert_model_url(preprocess_text)['pooled_output']

myoutputs=tf.keras.layers.Dense(128,activation='tanh',name='HiddenLayer1')(myoutputs)

myoutputs=tf.keras.layers.Dense(64,activation='tanh',name='HiddenLayer2')(myoutputs)

myoutputs=tf.keras.layers.Dense(1,activation='sigmoid',name='OutputLayer')(myoutputs)

model=tf.keras.Model(inputs=myinputs, outputs=myoutputs)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputLayer (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['InputLayer[0][0]']             
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [14]:
model.compile(loss='binary_crossentropy'
             ,optimizer='adam'
             ,metrics=['accuracy'])

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(dataFrame['Message'],dataFrame['Identifier'])

In [17]:
model.fit(x_train,y_train,epochs=1)

131/131 [==============================] - 615s 5s/step - loss: 0.2449 - accuracy: 0.9050


In [18]:
model.evaluate(x_test,y_test)

44/44 [==============================] - 204s 5s/step - loss: 0.1550 - accuracy: 0.9447


[0.15502092242240906, 0.9447236061096191]

In [19]:
#1.spam
#2.not a spam
#3. spam

check_predictions=[
    'XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL'
    ,'What you thinked about me. First time you saw me in class.'
    ,'As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a <E5><A3<A3>1500 Bonus Prize, call 09066364589'
]




In [20]:
model.predict(check_predictions)

array([[0.943959 ],
       [0.0053277],
       [0.7949698]], dtype=float32)

In [24]:
model.save('/Users/Bobbie/Downloads/BERT/mymodel/1')

INFO:tensorflow:Assets written to: /Users/Bobbie/Downloads/BERT/mymodel/1/assets


INFO:tensorflow:Assets written to: /Users/Bobbie/Downloads/BERT/mymodel/1/assets


In [25]:
model.save('/Users/Bobbie/Downloads/BERT/mymodel/2')

INFO:tensorflow:Assets written to: /Users/Bobbie/Downloads/BERT/mymodel/2/assets


INFO:tensorflow:Assets written to: /Users/Bobbie/Downloads/BERT/mymodel/2/assets
